In [1]:
import pandas

target_marker_names = [
"2_00148","2_07328"

]

target_chromosome_icim_binning = "Vu04(old11)" # chromosome to lookup in ICIM_binning file
target_chromosome_gff3 = "Vu04" # chormosome to lookup in ",
#Vu09(old8)
#Vu04(old11)

In [2]:
# load ICIM_binning dataset

icim_binning_df = pandas.read_excel( "ICIM_binning_CB27x125B.xlsx")
icim_binning_df.index = icim_binning_df["MarkerName"]

In [3]:
# extract all markers from the bins that contain the target marker names
result_headers = ["MarkerName", "BinID", "SNP position"]
result = {}
for header in result_headers :
    result[header] = []

for target_marker in target_marker_names:
    bin_id = icim_binning_df.loc[target_marker,"BinID"]
    if str(bin_id) == "0":
        icim_bin_subset = icim_binning_df[(icim_binning_df["MarkerName"] == target_marker) & (icim_binning_df["Chromosome"] == target_chromosome_icim_binning)]
    else:     
        icim_bin_subset = icim_binning_df[(icim_binning_df["BinID"] == bin_id) & (icim_binning_df["Chromosome"] == target_chromosome_icim_binning)]
    print( "for marker \"{0}\", found BinID \"{1}\", which contains a total of {2} markers" \
          .format( target_marker, bin_id, len(icim_bin_subset) ))
    
    for header in result_headers :
        result[header] += list(icim_bin_subset[header].values)
    
result_df = pandas.DataFrame( result )

for marker "2_00148", found BinID "161", which contains a total of 15 markers
for marker "2_07328", found BinID "458", which contains a total of 2 markers


In [4]:
# save results sorted by snp position
result_df.sort_values(["SNP position"]).drop_duplicates().to_csv("Vu04_60-60.93cM_MinInterval_SNPs_rqtl.csv", index=False)

In [5]:
result_df.sort_values

<bound method DataFrame.sort_values of    MarkerName  BinID  SNP position
0     2_00148    161    41216023.0
1     2_24504    161    41216486.0
2     2_13462    161    41222254.0
3     2_13461    161    41222366.0
4     2_25224    161    41249602.0
5     2_01776    161    41251676.0
6     2_01775    161    41251940.0
7     2_22280    161    41265751.0
8     2_16120    161    41273587.0
9     2_16121    161    41274353.0
10    2_45858    161    41278733.0
11    2_03714    161    41299078.0
12    2_52327    161    41320043.0
13    2_43314    161    41687276.0
14    2_27210    161    41754892.0
15    2_07328    458    42005753.0
16    2_16402    458    42012784.0>

In [6]:
# load gene annotation data to lookup more information
gff3_data = pandas.read_excel("Vunguiculata_469_v1.1.gene.gff3.xlsx", header=None)

# reduce the gff3_data to include only the target chromosome
gff3_data = gff3_data[gff3_data[0] == target_chromosome_gff3]

# find all the ID in gff3_data, between the minimum and maximmum SNP Position
result = {"ID":[]}
min_position = result_df["SNP position"].min()
max_position = result_df["SNP position"].max()
gff3_match = gff3_data[(gff3_data[3] <= max_position) & (gff3_data[4] >= min_position)].iloc[:,8].values
all_IDs = list(set([".".join(s.split( ";" )[0].split( "=" )[1].split(".")[:2]) for s in gff3_match]))

In [7]:
# filter out IDs that don't end with ".1"
filtered_IDs = [id for id in all_IDs if id.endswith(".1") ]

# put filtered IDs into a dataframe
result["ID"] = filtered_IDs
big_result_df = pandas.DataFrame( result )

In [8]:
big_result_df

,ID
0,Vigun04g187900.1
1,Vigun04g190000.1
2,Vigun04g193700.1
3,Vigun04g191600.1
4,Vigun04g193500.1
...,...
74,Vigun04g192200.1
75,Vigun04g189700.1
76,Vigun04g192500.1
77,Vigun04g189100.1


In [9]:
# all of these files have ID in the "transcriptName" column
gene_annotation_files = {
    "gene_annotation.xlsx": ["Pfam","Panther","KOG","ec","KO","GO",
                             "Best-hit-arabi-name","arabi-symbol","arabi-defline"],
    "Vunguiculata_469_v1.1.defline.xlsx": ["definition"]
}

for filename in gene_annotation_files.keys():
    # add gene annotation columns to result_df by looking up each ID
    gene_annotations = pandas.read_excel( filename )
    lookup_columns = gene_annotation_files[filename]
    for new_col in lookup_columns:
        big_result_df[new_col] = ""
    for row in big_result_df.index:
        ID = big_result_df.loc[row,"ID"]
        if ID is not None:
            match = gene_annotations[gene_annotations["transcriptName"] == ID]
            if len(match) > 0:
                big_result_df.loc[row,lookup_columns] = match.loc[match.index[0],lookup_columns]

In [12]:
# save results sorted by snp position
big_result_df.to_csv("Vu04_60-60.93cM_MinInterval_candidate_genes_rqtl.csv", index=False)

In [11]:
big_result_df

,ID,Pfam,Panther,KOG,ec,KO,GO,Best-hit-arabi-name,arabi-symbol,arabi-defline,definition
0,Vigun04g187900.1,"PF08240,PF13602","PTHR11695,PTHR11695:SF530",KOG1198,1.3.1.74,K18980,"GO:0055114,GO:0016491,GO:0008270",AT1G23740.1,NaN,"Oxidoreductase, zinc-binding dehydrogenase fam...",(1 of 7) K18980 - 2-methylene-furan-3-one redu...
1,Vigun04g190000.1,PF00931,"PTHR23155,PTHR23155:SF414",KOG4658,NaN,NaN,GO:0043531,AT3G14470.1,NaN,NB-ARC domain-containing disease resistance pr...,(1 of 78) PTHR23155//PTHR23155:SF414 - LEUCINE...
2,Vigun04g193700.1,PF01073,"PTHR10366,PTHR10366:SF388",KOG1502,1.2.1.44,NaN,"GO:0055114,GO:0016616,GO:0006694,GO:0003854",AT5G14700.1,NaN,NAD(P)-binding Rossmann-fold superfamily protein,(1 of 1) PTHR10366:SF388 - CINNAMOYL COA REDUC...
3,Vigun04g191600.1,PF00931,"PTHR23155,PTHR23155:SF414",KOG4658,NaN,NaN,GO:0043531,AT3G14470.1,NaN,NB-ARC domain-containing disease resistance pr...,(1 of 78) PTHR23155//PTHR23155:SF414 - LEUCINE...
4,Vigun04g193500.1,"PF00122,PF12710,PF16209,PF16212","PTHR24092,PTHR24092:SF75",NaN,3.6.3.1,K01530,"GO:0046872,GO:0000166,GO:0016021,GO:0015914,GO...",AT3G27870.1,NaN,ATPase E1-E2 type family protein / haloacid de...,(1 of 2) PTHR24092:SF75 - PHOSPHOLIPID-TRANSPO...
...,...,...,...,...,...,...,...,...,...,...,...
74,Vigun04g192200.1,PF00931,"PTHR23155,PTHR23155:SF414",KOG4658,NaN,NaN,GO:0043531,AT3G14470.1,NaN,NB-ARC domain-containing disease resistance pr...,(1 of 78) PTHR23155//PTHR23155:SF414 - LEUCINE...
75,Vigun04g189700.1,NaN,"PTHR23155,PTHR23155:SF414",NaN,NaN,NaN,NaN,AT3G14470.1,NaN,NB-ARC domain-containing disease resistance pr...,(1 of 78) PTHR23155//PTHR23155:SF414 - LEUCINE...
76,Vigun04g192500.1,PF00931,"PTHR23155,PTHR23155:SF414",KOG4658,NaN,NaN,GO:0043531,AT3G14470.1,NaN,NB-ARC domain-containing disease resistance pr...,(1 of 78) PTHR23155//PTHR23155:SF414 - LEUCINE...
77,Vigun04g189100.1,"PF00690,PF00702,PF00122","PTHR24093,PTHR24093:SF355",NaN,3.6.3.6,K01535,"GO:0046872,GO:0000166",AT1G80660.1,"AHA9,HA9",H(+)-ATPase 9,"(1 of 4) PTHR24093:SF355 - ATPASE 1, PLASMA ME..."
